# 작사가 인공지능 만들기
## 순서
1. 필요한 모듈, 패키지 import
2. 필요한 함수 선언
3. 데이터 불러오기
4. 데이터 정제
5. 데이터셋 분리
6. 모델 생성 및 훈련
7. 가사 생성
8. 회고

### 1. 필요한 모듈, 패키지 import

In [1]:
import glob, os, re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

### 2. 필요한 함수, 클래스 선언

In [2]:
def preprocess_sentence(sentence):
    '''
      단어 정제를 위한 함수
    '''
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [40]:
def tokenize(corpus):
    '''
      Tokenizer 생성 함수
      tokenizer과 tensor을 반환
    '''
    # 12000단어를 기억할 수 있는 tokenizer를 생성
    # 우리는 이미 문장을 정제했으니 filters는 불필요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀 것
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줌
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [62]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    '''
      조건에 맞게 문장을 생성해 반환
    '''
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

### 3. 데이터 불러오기

In [41]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### 4. 데이터 정제

In [42]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [43]:
tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f938054f8e0>


### 6. 데이터 셋 분리

**조건**
- 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거
- 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외

In [44]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1: ]

In [45]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2)

In [46]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [57]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

In [59]:
# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 생성
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [58]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(256, drop_remainder=True)

### 7. 모델 생성 및 훈련

In [ ]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [61]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
686/686 [==============================] - 128s 181ms/step - loss: 3.6339 - val_loss: 3.2298
Epoch 2/10
686/686 [==============================] - 125s 181ms/step - loss: 3.1311 - val_loss: 2.9754
Epoch 3/10
686/686 [==============================] - 125s 182ms/step - loss: 2.9349 - val_loss: 2.8046
Epoch 4/10
686/686 [==============================] - 125s 182ms/step - loss: 2.7846 - val_loss: 2.6656
Epoch 5/10
686/686 [==============================] - 125s 182ms/step - loss: 2.6524 - val_loss: 2.5326
Epoch 6/10
686/686 [==============================] - 125s 182ms/step - loss: 2.5320 - val_loss: 2.4152
Epoch 7/10
686/686 [==============================] - 125s 182ms/step - loss: 2.4204 - val_loss: 2.3033
Epoch 8/10
686/686 [==============================] - 125s 182ms/step - loss: 2.3165 - val_loss: 2.2012
Epoch 9/10
686/686 [==============================] - 125s 182ms/step - loss: 2.2162 - val_loss: 2.1010
Epoch 10/10
686/686 [==============================] - 125s 182m

### 7. 가사 생성

In [74]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you lie <end> '

### 8. 회고
embedding_size와 hidden_size의 값이 클 수록 모델의 성능이 올라갔지만, 일정 수준 이상에서는 오히려 혼란을 증가시킬 수 있다는 것을 알았다. 
init_sentence에서 단어장에 없는 단어를 넣을 경우 <unk>로만 출력되는 것을 확인할 수 있었다.